# Load data

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('clean_year_text_keywords.csv')
df.head()

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,year,text,keywords
0,24877594,2014.0,What we know and do not know about the cannab...,"['cannabinoid receptor 2', 'endocannabinoid', ..."
1,16889837,2006.0,Allosteric agonists of 7TM receptors: expandi...,NaN
2,31068464,2019.0,MRGPRX4 is a G protein-coupled receptor activ...,"['mrgprx4', 'bile acids', 'cholestasis', 'itch..."
3,30610192,2019.0,"Modulation of Gpr39, a G-protein coupled rece...",NaN
4,30289386,2018.0,Simulation of spontaneous G protein activatio...,"['g proteins', 'markov state models', 'alloste..."


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323870 entries, 0 to 323869
Data columns (total 4 columns):
Id          323870 non-null int64
year        323603 non-null float64
text        323870 non-null object
keywords    37845 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 9.9+ MB


# Build vocabulary

## normalize text

In [11]:
import spacy
import nltk
import re
import unicodedata

nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)

In [12]:
def lemmatize_text(text): # normalize word variant forms to the same word
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

lemmatize_text("My system keeps crashing! his crashed yesterday, ours crashes daily")

'My system keep crash ! his crashed yesterday , ours crash daily'

In [21]:
df.text = df.text.apply(lambda x: x.lower())

In [23]:
df.text = df.text.apply(lemmatize_text) # normalize word variants

In [36]:
df.text = df.text.apply(lambda x: re.sub(r' +',' ',x)) # remove excessive space

In [88]:
def remove_puncta(text):
    pattern = r'[^a-z0-9\s\.]'
    text = re.sub(pattern, '', text)
    return text

In [89]:
df.text = df.text.apply(remove_puncta) # remove puncta

## Tokenize to sentence to create corpus

In [90]:
corpus = []
for i in range(df.text.shape[0]):
    corpus += df.text.iloc[i].strip().split('.')

In [91]:
with open('corpus.txt','w') as f:
    for i in range(len(corpus)):
        f.write(corpus[i])
        f.write('.')
        f.write('\n')

## Phrase modeling

In [80]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence
import itertools as it
import os

In [83]:
currentdir = os.getcwd()
def datadir(file):
    return currentdir+'/'+file
datadir('corpus.txt')

'/Users/shunling/Desktop/SpringBoard/CapstoneProject2/corpus.txt'

### from unigram to bigram

In [92]:
unigrams = LineSentence(datadir('corpus.txt'))
for s in it.islice(unigrams, 10,12):
    print(' '.join(s))
    print('')
for sentence in unigrams:
    print(sentence)
    break

allosteric agonist of 7tm receptor expand the pharmacological toolbox .

abstract approximately 1 of the genome of high organism encode seven transmembrane 7tm g protein couple receptor which control an extensive range of physiological process and represent drug target for nearly half of all drug that be prescribe currently .

['what', 'we', 'know', 'and', 'do', 'not', 'know', 'about', 'the', 'cannabinoid', 'receptor', '2', 'cb2', '.']


In [93]:
%%time
bigram_phrases = Phrases(unigrams) # train model
bigram_model = Phraser(bigram_phrases) # build model

CPU times: user 3min 40s, sys: 4.13 s, total: 3min 44s
Wall time: 3min 48s


In [94]:
for s in it.islice(unigrams, 10,12):
    print(' '.join(bigram_model[s]))
    print('')

allosteric agonist of 7tm receptor expand the pharmacological toolbox .

abstract approximately 1 of the genome of high organism encode seven_transmembrane 7tm g_protein couple receptor which control an extensive range of physiological_process and represent drug target for nearly_half of all drug that be prescribe currently .



In [98]:
%%time
with open('bigrams.txt','w') as f:
    for sent in unigrams:
        bigram_sent = ' '.join(bigram_model[sent]) # apply model
        f.write(bigram_sent)
        f.write('\n')

CPU times: user 3min 37s, sys: 1.71 s, total: 3min 39s
Wall time: 3min 40s


### from bigram to trigram

In [102]:
%%time
bigrams = LineSentence(datadir('bigrams.txt'))
trigram_phrases = Phrases(bigrams)
trigram_model = Phraser(trigram_phrases)
with open('trigrams.txt','w') as f:
    for sent in bigrams:
        tri_sent = ' '.join(trigram_model[sent])
        f.write(tri_sent)
        f.write('\n')

CPU times: user 7min 38s, sys: 4.04 s, total: 7min 42s
Wall time: 7min 46s


### from trigram to fourgram

In [106]:
%%time
trigrams = LineSentence(datadir('trigrams.txt'))
fourgram_phrases = Phrases(trigrams)
fourgram_model = Phraser(fourgram_phrases)
with open('fourgrams.txt','w') as f:
    for sent in trigrams:
        four_sent = ' '.join(fourgram_model[sent])
        f.write(four_sent)
        f.write('\n')

CPU times: user 8min 1s, sys: 4.67 s, total: 8min 5s
Wall time: 8min 10s


# Topic modeling with LDA with BOW

In [111]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle 
warnings.simplefilter("ignore")

In [110]:
%%time
text = LineSentence(datadir('fourgrams.txt'))
dictionary = Dictionary(text)# bag of word, Id2word

CPU times: user 2min 36s, sys: 3.2 s, total: 2min 39s
Wall time: 2min 52s


In [113]:
dictionary.filter_extremes(no_above=0.4) 
dictionary.compactify()
dictionary.save(datadir('fourgram_dict_all.dict'))

In [114]:
def bow_generator(file):
    for text in LineSentence(file):
        yield dictionary.doc2bow(text)

In [117]:
MmCorpus.serialize(datadir('bow_corpus_all.mm'),bow_generator(datadir('fourgrams.txt')))

In [118]:
bow_corpus = MmCorpus(datadir('bow_corpus_all.mm')) # word count vectorizer

In [119]:
%%time
lda = LdaMulticore(bow_corpus,
                  num_topics = 50,
                  id2word = dictionary,
                  workers = 3)
lda.save(datadir('lda50'))

CPU times: user 29min 6s, sys: 13min 32s, total: 42min 39s
Wall time: 36min 34s


In [131]:
def explore_topic(topic_n, topn=25):
    print('{:30} {}'.format('term','frequency') + '\n')
    
    for term, frequency in lda.show_topic(topic_n, topn=25):
        print('{:30} {:.3f}'.format(term, round(frequency, 3)))

In [158]:
explore_topic(29)

term                           frequency

e.                             0.053
regulate                       0.050
mean                           0.041
neuronal                       0.040
cell                           0.040
endothelin                     0.038
ca2i                           0.037
growth                         0.037
cellular                       0.035
factor                         0.034
directly                       0.027
andor                          0.024
by                             0.024
angiotensin                    0.019
differentiation                0.016
generation                     0.016
26                             0.015
ca                             0.015
second_messenger               0.011
be                             0.010
function                       0.009
uterus                         0.009
fibroblast                     0.009
output                         0.009
that                           0.008


In [159]:
topics = {18:'neuroscience',19:'antidepressant',1:'beta receptor',7:'opioid',12:'pharmacology'}

In [160]:
%%time
lda = LdaMulticore(bow_corpus,
                  num_topics = 10,
                  id2word = dictionary,
                  workers = 3)
lda.save(datadir('lda10'))

CPU times: user 13min 47s, sys: 3min 10s, total: 16min 58s
Wall time: 17min 52s


In [173]:
explore_topic(9)

term                           frequency

0.                             0.082
1                              0.062
2                              0.050
3                              0.041
5                              0.037
4                              0.032
6                              0.020
nm                             0.020
day                            0.019
respectively                   0.016
7                              0.016
2.                             0.016
8                              0.014
for                            0.013
at                             0.012
p_0.                           0.010
be                             0.010
0                              0.010
01                             0.009
3.                             0.009
cell                           0.008
n                              0.008
10                             0.008
bmax                           0.007
microm                         0.007


## Neural Network Vectorization (Word2Vec)

In [192]:
import gensim
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import warnings 

warnings.filterwarnings(action = 'ignore') 

In [213]:
for s in it.islice(text, 0,10):
    print(s[:-1])

['what', 'we', 'know', 'and', 'do_not', 'know_about', 'the', 'cannabinoid', 'receptor', '2', 'cb2']
['abstract', 'it', 'have', 'be', 'well_appreciate', 'that', 'the', 'endocannabinoid_system', 'can', 'regulate', 'immune_response', 'via', 'the', 'cannabinoid', 'receptor', '2', 'cb2', 'which', 'be', 'primarily', 'express', 'by', 'cell', 'of', 'the', 'hematopoietic', 'system']
['the', 'endocannabinoid_system', 'be', 'compose', 'of', 'receptor', 'ligand', 'and', 'enzyme', 'control', 'the', 'synthesis', 'and', 'degradation', 'of', 'endocannabinoid']
['along', 'with', 'endocannabinoid', 'both', 'plant_derive', 'and', 'synthetic_cannabinoid', 'have', 'be', 'show', 'to', 'bind', 'to', 'and', 'signal', 'through', 'cb2', 'via', 'g_protein', 'lead', 'to', 'both', 'inhibitory', 'and', 'stimulatory', 'signal', 'depend_on', 'the', 'biological_process']
['because', 'no', 'cannabinoid', 'ligand', 'have', 'be', 'identify', 'that', 'only', 'bind', 'to', 'cb2', 'the', 'generation', 'of', 'mouse_deficient

In [238]:
def sentenize(doc):
    new_doc = []
    abstract = []
    for sent in doc:
        
        if sent != ['.']:
            if sent[0] == 'abstract':
                abstract += sent[1:-1]
            else:
                abstract += sent[:-1]
        else:
            new_doc.append(abstract)
            abstract = []

    return new_doc

In [239]:
%%time
abstracts = sentenize(text)

CPU times: user 30 s, sys: 24.6 s, total: 54.6 s
Wall time: 1min 17s


In [243]:
print(len(abstracts))
print(abstracts[0:1])

325620
[['what', 'we', 'know', 'and', 'do_not', 'know_about', 'the', 'cannabinoid', 'receptor', '2', 'cb2', 'it', 'have', 'be', 'well_appreciate', 'that', 'the', 'endocannabinoid_system', 'can', 'regulate', 'immune_response', 'via', 'the', 'cannabinoid', 'receptor', '2', 'cb2', 'which', 'be', 'primarily', 'express', 'by', 'cell', 'of', 'the', 'hematopoietic', 'system', 'the', 'endocannabinoid_system', 'be', 'compose', 'of', 'receptor', 'ligand', 'and', 'enzyme', 'control', 'the', 'synthesis', 'and', 'degradation', 'of', 'endocannabinoid', 'along', 'with', 'endocannabinoid', 'both', 'plant_derive', 'and', 'synthetic_cannabinoid', 'have', 'be', 'show', 'to', 'bind', 'to', 'and', 'signal', 'through', 'cb2', 'via', 'g_protein', 'lead', 'to', 'both', 'inhibitory', 'and', 'stimulatory', 'signal', 'depend_on', 'the', 'biological_process', 'because', 'no', 'cannabinoid', 'ligand', 'have', 'be', 'identify', 'that', 'only', 'bind', 'to', 'cb2', 'the', 'generation', 'of', 'mouse_deficient', 'in',

In [193]:
class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.loss_s = 0
        
    def on_epoch_end(self,model):
            
        if self.epoch % 10 == 0:
            loss_e = int(model.get_latest_training_loss())
            loss = loss_e - self.loss_s
            self.loss_s = loss_e
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1     

In [244]:
%%time
feature_size = 100 # word vector dimensionality
window_context = 30 # context window size
min_word_count = 5 # minimum word count
sample = 1e-3 # Downsample setting for frequent words
model1 = Word2Vec(abstracts, #fourgrams abstract list tokenized in words
                 size = feature_size,
                 window = window_context,
                 min_count = min_word_count,
                 sample = sample,
                 sg=1,
                 compute_loss = True,
                 iter = 100,
                 workers = 3,
                 callbacks=[callback()]
                 )

KeyboardInterrupt: 

In [ ]:
# view similar words based on gensim's model
similar_words = {search_term: [item[0] for item in model1.wv.most_similar([search_term], topn=7)]
                  for search_term in ['g_protein_couple','dopamine','mu','']}
similar_words